In [9]:
import numpy as np
import matplotlib.pyplot as plt
from spectral import *
from scipy import linalg
import scipy.stats as stats
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from scipy.spatial.distance import euclidean
from scipy.stats import f_oneway
import pandas as pd
import cv2
import glob
import os

In [10]:
class DataProcessor:
    def __init__(self, msi_data_dir, msi_masks_dir, msi_rgbs_dir):
                  
        self.msi_data_dir = msi_data_dir
        self.msi_masks_dir = msi_masks_dir
        self.msi_rgbs_dir = msi_rgbs_dir

        self.msi_imgs = []
        self.msi_masks = []
        self.msi_rgbs = []
        self.msi_data_name = []
        self.msi_masks_name = []
        self.msi_rgbs_name = []

    def load_multispectral_data(self, data_dir, imgs_list, data_name):
        # msi_files = data_dir 
        msi_files = glob.glob(os.path.join(data_dir))
        for msi_file in msi_files:
            img_data = open_image(msi_file)
            imgs_list.append(np.asarray(img_data.load()))
            img_filename = os.path.basename(msi_file)
            data_name.append(img_filename)

    def load_masks(self, masks_dir, masks_list, masks_name):
        # mask_files = masks_dir
        mask_files = glob.glob(os.path.join(masks_dir))
        for mask_file in mask_files:
            mask_data = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
            _, mask_data = cv2.threshold(mask_data, 0, 1, cv2.THRESH_BINARY)
            masks_list.append(mask_data)
            mask_filename = os.path.basename(mask_file)
            masks_name.append(mask_filename)

    def load_rgbs(self, rgbs_dir, rgbs_list, rgbs_name):
        # rgb_files = rgbs_dir
        rgb_files = glob.glob(os.path.join(rgbs_dir))
        for rgb_file in rgb_files:
            rgb_im = cv2.imread(rgb_file)
            rgb_im = cv2.cvtColor(rgb_im, cv2.COLOR_BGR2RGB)
            rgbs_list.append(rgb_im)
            rgb_filename = os.path.basename(rgb_file)
            rgbs_name.append(rgb_filename)

    def process_data(self):
        self.load_multispectral_data(self.msi_data_dir, self.msi_imgs, self.msi_data_name)
        self.load_masks(self.msi_masks_dir, self.msi_masks, self.msi_masks_name)
        self.load_rgbs(self.msi_rgbs_dir, self.msi_rgbs, self.msi_rgbs_name)

In [11]:
# Create an instance of the DataProcessor class and process the data
base_dir = "C:/Users/htic/Documents/MSI_dataset_Copy/"

data_processor = DataProcessor(

    #Read all the data
    msi_data_dir =  os.path.join(base_dir, "data/*/*.hdr"), 
    msi_masks_dir = os.path.join(base_dir, "masks/*.png"),
    msi_rgbs_dir =  os.path.join(base_dir, "rgb/*.png")

)
data_processor.process_data()


In [12]:
# Custom sorting function to extract the numerical part of the filename
def sort_key(filename):
    return int(''.join(filter(str.isdigit, filename)))

# Sort the filenames using the custom sorting function
sorted_data_names = sorted(data_processor.msi_data_name, key=sort_key)
sorted_mask_names = sorted(data_processor.msi_masks_name, key=sort_key)
sorted_rgb_names = sorted(data_processor.msi_rgbs_name, key=sort_key)

# Print the sorted filenames
for i, (data_name, mask_name, rgb_name) in enumerate(zip(sorted_data_names, sorted_mask_names, sorted_rgb_names)):
    print(f"{i}. Data = {data_name}, Mask = {mask_name}, RGB = {rgb_name}")


0. Data = 1.hdr, Mask = 1.png, RGB = 1.png
1. Data = 2.hdr, Mask = 2.png, RGB = 2.png
2. Data = 3.hdr, Mask = 3.png, RGB = 3.png
3. Data = 4.hdr, Mask = 4.png, RGB = 4.png
4. Data = 6.hdr, Mask = 6.png, RGB = 5.png
5. Data = 7.hdr, Mask = 7.png, RGB = 6.png
6. Data = 8.hdr, Mask = 8.png, RGB = 7.png
7. Data = 9.hdr, Mask = 9.png, RGB = 8.png
8. Data = 10.hdr, Mask = 10.png, RGB = 9.png
9. Data = 12.hdr, Mask = 12.png, RGB = 10.png
10. Data = 13.hdr, Mask = 13.png, RGB = 11.png
11. Data = 14.hdr, Mask = 14.png, RGB = 12.png
12. Data = 15.hdr, Mask = 15.png, RGB = 13.png
13. Data = 16.hdr, Mask = 16.png, RGB = 14.png
14. Data = 17.hdr, Mask = 17.png, RGB = 15.png
15. Data = 18.hdr, Mask = 18.png, RGB = 16.png
16. Data = 19.hdr, Mask = 19.png, RGB = 17.png
17. Data = 20.hdr, Mask = 20.png, RGB = 18.png
18. Data = 21.hdr, Mask = 21.png, RGB = 19.png
19. Data = 22.hdr, Mask = 22.png, RGB = 20.png
20. Data = 23.hdr, Mask = 23.png, RGB = 21.png
21. Data = 26.hdr, Mask = 26.png, RGB = 22.png
2

In [13]:
for i in range(len(data_processor.msi_imgs)):
    print(f'{i}. Image:{data_processor.msi_data_name[i]} \t size: {data_processor.msi_imgs[i].shape}')

0. Image:1.hdr 	 size: (270, 510, 16)
1. Image:10.hdr 	 size: (270, 510, 16)
2. Image:100.hdr 	 size: (270, 510, 16)
3. Image:103.hdr 	 size: (270, 510, 16)
4. Image:104.hdr 	 size: (270, 510, 16)
5. Image:105.hdr 	 size: (270, 510, 16)
6. Image:106.hdr 	 size: (270, 510, 16)
7. Image:109.hdr 	 size: (270, 510, 16)
8. Image:110.hdr 	 size: (270, 510, 16)
9. Image:111.hdr 	 size: (270, 510, 16)
10. Image:112.hdr 	 size: (270, 510, 16)
11. Image:113.hdr 	 size: (270, 510, 16)
12. Image:115.hdr 	 size: (270, 510, 16)
13. Image:116.hdr 	 size: (270, 510, 16)
14. Image:117.hdr 	 size: (270, 510, 16)
15. Image:118.hdr 	 size: (270, 510, 16)
16. Image:119.hdr 	 size: (270, 510, 16)
17. Image:12.hdr 	 size: (270, 510, 16)
18. Image:120.hdr 	 size: (270, 510, 16)
19. Image:121.hdr 	 size: (270, 510, 16)
20. Image:123.hdr 	 size: (270, 510, 16)
21. Image:124.hdr 	 size: (270, 510, 16)
22. Image:125.hdr 	 size: (270, 510, 16)
23. Image:126.hdr 	 size: (270, 510, 16)
24. Image:127.hdr 	 size: (270

In [14]:
def oxy_cal(img):
   # R542 = 52096 , 49708
   # R572 = 49172, 43340 
    attmat = np.array([[52096,49708],[49172,43340]])

    lnR544 = np.log(img[:,:,10])  #changing reflectance to absorbance
    lnR572 = np.log(img[:,:,14])  #800

    has_inf = np.isinf(lnR544).any()

    if has_inf:
        print("The image contains inf values.")
 
    lnR544 = np.nan_to_num(lnR544)
    lnR572 = np.nan_to_num(lnR572)

    lnR544 = lnR544.reshape(270*510)
    lnR572 = lnR572.reshape(270*510)
    
    y = np.linalg.lstsq(attmat, [lnR544, lnR572])[0]
    hbo2 = y[0]
    hb = y[1]
    hbO2 = hbo2.reshape(270,510)
    Hbb = hb.reshape(270,510)

    n_Hb = ((Hbb- Hbb.min())/(Hbb.max()-Hbb.min())) * 100
    n_HbO2 = ((hbO2- hbO2.min())/(hbO2.max()-hbO2.min())) * 100

    StO2 = n_HbO2/(n_Hb+n_HbO2)
    n_StO2 = ((StO2 - StO2.min()))/(StO2.max() - StO2.min()) * 100        
    return n_Hb, n_HbO2, n_StO2

In [15]:
def analyser(rgbs, imgs, masks, data_names):
    mean_StO2 = []
    
    for i in range(len(imgs)):
        image = imgs[i]
        mask = masks[i]
        rgb = rgbs[i]

        Hb, HbO2, StO2 = oxy_cal(image)
        masked_data = cv2.bitwise_and(StO2, StO2, mask=mask)
        
        unmasked_pixels = masked_data[np.nonzero(masked_data)]
        mean_StO2.append(np.mean(unmasked_pixels))
    
    return mean_StO2, data_names

In [16]:
#Calculating the StO2 and generating spectral graph - Leuko
image_mean_StO2, data_names = analyser(data_processor.msi_rgbs, data_processor.msi_imgs, data_processor.msi_masks, data_processor.msi_data_name)

C:\Users\htic\AppData\Local\Temp\ipykernel_25468\4175512468.py:6: RuntimeWarning: invalid value encountered in log
  lnR544 = np.log(img[:,:,10])  #changing reflectance to absorbance
C:\Users\htic\AppData\Local\Temp\ipykernel_25468\4175512468.py:7: RuntimeWarning: invalid value encountered in log
  lnR572 = np.log(img[:,:,14])  #800
C:\Users\htic\AppData\Local\Temp\ipykernel_25468\4175512468.py:20: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  y = np.linalg.lstsq(attmat, [lnR544, lnR572])[0]


In [17]:
result_df = pd.DataFrame({'Data Name': data_names, 'StO2': image_mean_StO2})

# Save the DataFrame to an Excel file
writer = pd.ExcelWriter("C:/Users/htic/Documents/Jupyter/MSI_Oral_Analysis/StO2_all_data.xlsx")
result_df.to_excel(writer, sheet_name="StO2", index=False)
writer.save()
